In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from datasets import load_dataset, DatasetDict

In [3]:
# ======= 3️⃣ Load dataset =======
dataset = load_dataset("twice123nd/Dataset_summarization")
data = dataset["train"]

print("Tổng số mẫu gốc:", len(data))


README.md:   0%|          | 0.00/234 [00:00<?, ?B/s]

sample1.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Tổng số mẫu gốc: 2986


In [4]:

print("Tổng số mẫu train:", len(data))

# In 3 sample đầu tiên
for i in range(3):
    sample = data[i]
    print(f"\n--- Sample {i+1} ---")
    for col_name, value in sample.items():
        # Nếu giá trị là string dài, chỉ in 300 ký tự đầu
        if isinstance(value, str) and len(value) > 300:
            print(f"{col_name}: {value[:300]} ...")
        else:
            print(f"{col_name}: {value}")

Tổng số mẫu train: 2986

--- Sample 1 ---
title: Lý giải việc nhiều vụ kiện chủ tịch tỉnh ra tòa chưa được thi hành án
document: Bộ trưởng Tư pháp Lê Thành Long vừa có báo cáo gửi Quốc hội về việc thực hiện nghị quyết giám sát chuyên đề, chất vấn thuộc lĩnh vực tư pháp.Trong lĩnh vực theo dõi thi hành án hành chính, Bộ trưởng Tư pháp cho biết từ 1/10/2023 đến hết tháng 3/2024, tòa án các cấp đã chuyển giao cho các cơ quan thi ...
summary: Theo Bộ Tư pháp, còn nhiều bản án, quyết định của tòa về vụ án hành chính đã có hiệu lực nhưng chưa được thi hành xong, trong đó không ít bản án người phải thi hành án là UBND, chủ tịch UBND cấp tỉnh.

--- Sample 2 ---
title: Vĩnh Phúc trình Chính phủ sáp nhập 28 đơn vị hành chính cấp xã
document: Theo lãnh đạo Sở Nội vụ tỉnh Vĩnh Phúc, từ nay tới năm 2025 không có đơn vị hành chính cấp huyện nào ở tỉnh thuộc diện phải sắp xếp, sáp nhập.Vĩnh Phúc có 6 huyện, thành phố có đơn vị hành chính cấp xã phải sắp xếp, gồm: Tam Dương, Yên Lạc, Vĩnh Tường, Sông 

In [5]:
from datasets import DatasetDict

# Tách 10% làm validation, 10% làm test
split_val_test = data.train_test_split(test_size=0.2, seed=42)  # 20% = val+test
val_test = split_val_test['test'].train_test_split(test_size=0.5, seed=42)  # chia đều val & test

dataset = DatasetDict({
    'train': split_val_test['train'],
    'validation': val_test['train'],
    'test': val_test['test']
})

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['title', 'document', 'summary'],
        num_rows: 2388
    })
    validation: Dataset({
        features: ['title', 'document', 'summary'],
        num_rows: 299
    })
    test: Dataset({
        features: ['title', 'document', 'summary'],
        num_rows: 299
    })
})


In [6]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

model_name = "VietAI/vit5-base" # hoặc "vinai/bartpho-summarization"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_input_length = 1024
max_target_length = 512

def preprocess_function(batch):
    # Kết hợp title + document làm input
    inputs = [t + ": " + d for t, d in zip(batch["title"], batch["document"])]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Summary làm target
    labels = tokenizer(batch["summary"], max_length=max_target_length, truncation=True).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

2025-11-19 10:52:06.386710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763549526.571563      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763549526.621135      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/2388 [00:00<?, ? examples/s]

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

In [7]:
# Kiểm tra tokenized sample đầu tiên
sample = tokenized_dataset["train"][0]

print("Các key có trong tokenized sample:", sample.keys())

# In vài token đầu để kiểm tra
print("\nInput_ids (đầu 20 tokens):", sample["input_ids"][:20])
print("Attention_mask (đầu 20 tokens):", sample["attention_mask"][:20])
print("Labels (đầu 20 tokens):", sample["labels"][:20])

Các key có trong tokenized sample: dict_keys(['title', 'document', 'summary', 'input_ids', 'attention_mask', 'labels'])

Input_ids (đầu 20 tokens): [122, 1353, 1269, 954, 1281, 3500, 382, 1541, 3395, 1245, 35862, 8375, 1855, 897, 745, 1320, 3958, 189, 35836, 35849]
Attention_mask (đầu 20 tokens): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Labels (đầu 20 tokens): [1399, 93, 35790, 1056, 210, 1670, 424, 900, 400, 71, 1071, 68, 235, 498, 382, 890, 35790, 871, 6898, 123]


In [7]:
print("== Input Text ==")
print(tokenizer.decode(sample["input_ids"], skip_special_tokens=True))

print("\n== Summary ==")
print(tokenizer.decode(sample["labels"], skip_special_tokens=True))

print("\n== Attention Mask (độ dài):", len(sample["attention_mask"]))
print("== Input IDs (đầu 20):", sample["input_ids"][:20])

== Input Text ==
3 câu chuyện truyền cảm hứng chiến thắng ung thư: Cơn bạo bệnh bất ngờ ập đếnCứ nhắc tới hai chữ "ung thư" là người ta nghĩ ngay đến "án tử". Bệnh ung thư quái ác dường như đến với con người ta như một cơn bão, không được dự báo trước nên đã khiến nhiều người rơi vào tình cảnh tuyệt vọng và mất niềm tin vào cuộc sống.VTV2 HTCB số 23 - Hành trình tìm lại sự sống của bệnh nhân ung thư Vòm họngCũng giống như bao người đang mang trong mình căn bệnh hiểm nghèo này, kể về thời điểm bắt đầu phát hiện bệnh từ những cơn đau đầu không rõ nguyên nhân, Ông Trần Văn Tiến (Hà Nam) đi khám và uống thuốc Đông y 6 tháng mà không đỡ. Cho đến tận năm 2018, khi các cơn đau ngày càng nặng hơn, ông quyết định đến Bệnh viện Bạch Mai khám, ở đó bác sĩ thông báo ông bị Ung thư vòm họng giai đoạn 3.Chân dung 3 nhân vật truyền cảm hứng chiến thắng ung thư.Cơn bão ung thư cũng xảy đến với ông Vũ Huy Chương (xã Kim Hải, huyện Kim Sơn, Ninh Bình) vào năm 2002, khi cảm thấy sức khỏe yếu đi rõ rệt, đ

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,  # rất quan trọng với T5/BART để xử lý label padding đúng
    return_tensors="pt"
)


In [9]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./output_loss_only",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,

    eval_strategy="steps",
    eval_steps=100,                  
    logging_strategy="steps",
    logging_steps=50,               

    save_strategy="no",              
    predict_with_generate=True,    
    report_to="none",                
    fp16=True
)


In [10]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,               # vẫn giữ để dùng decode, compute_metrics
    data_collator=data_collator       # ✅ THAY VÌ dùng tokenizer nội bộ
)


/tmp/ipykernel_48/2783352374.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [11]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.957400,1.741829
200,1.717300,1.702770
300,1.684100,1.683081
400,1.567700,1.682881


TrainOutput(global_step=450, training_loss=1.7933304002549912, metrics={'train_runtime': 1716.9425, 'train_samples_per_second': 4.173, 'train_steps_per_second': 0.262, 'total_flos': 8516090557624320.0, 'train_loss': 1.7933304002549912, 'epoch': 3.0})

In [12]:
trainer.save_model("./output_loss_only")           # lưu model weights
tokenizer.save_pretrained("./output_loss_only")    # lưu tokenizer


('./output_loss_only/tokenizer_config.json',
 './output_loss_only/special_tokens_map.json',
 './output_loss_only/spiece.model',
 './output_loss_only/added_tokens.json',
 './output_loss_only/tokenizer.json')

In [53]:
!pip install evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 38.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pyli

In [52]:
!pip install bert_score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nv

In [70]:
for i, (p, r, f) in enumerate(zip(results['precision'], results['recall'], results['f1']), 1):
    print(f"Sample {i:2d} | Precision: {p:.3f} | Recall: {r:.3f} | F1: {f:.3f}")
# Tính trung bình
avg_precision = sum(results['precision']) / len(results['precision'])
avg_recall = sum(results['recall']) / len(results['recall'])
avg_f1 = sum(results['f1']) / len(results['f1'])

print("\nAverage | Precision: {:.3f} | Recall: {:.3f} | F1: {:.3f}".format(
    avg_precision, avg_recall, avg_f1
))

Sample  1 | Precision: 0.652 | Recall: 0.686 | F1: 0.669
Sample  2 | Precision: 0.638 | Recall: 0.743 | F1: 0.687
Sample  3 | Precision: 0.707 | Recall: 0.791 | F1: 0.747
Sample  4 | Precision: 0.696 | Recall: 0.742 | F1: 0.718
Sample  5 | Precision: 0.679 | Recall: 0.747 | F1: 0.712
Sample  6 | Precision: 0.678 | Recall: 0.730 | F1: 0.703
Sample  7 | Precision: 0.683 | Recall: 0.782 | F1: 0.729
Sample  8 | Precision: 0.655 | Recall: 0.720 | F1: 0.686
Sample  9 | Precision: 0.712 | Recall: 0.756 | F1: 0.733
Sample 10 | Precision: 0.720 | Recall: 0.785 | F1: 0.751
Sample 11 | Precision: 0.636 | Recall: 0.678 | F1: 0.656
Sample 12 | Precision: 0.647 | Recall: 0.711 | F1: 0.677
Sample 13 | Precision: 0.694 | Recall: 0.708 | F1: 0.701
Sample 14 | Precision: 0.667 | Recall: 0.731 | F1: 0.697
Sample 15 | Precision: 0.653 | Recall: 0.725 | F1: 0.687
Sample 16 | Precision: 0.666 | Recall: 0.743 | F1: 0.702
Sample 17 | Precision: 0.708 | Recall: 0.741 | F1: 0.724
Sample 18 | Precision: 0.679 | 

In [16]:
import openai
import os
import json
import time

In [17]:
!pip install --upgrade openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.3 MB/s eta 0:00:0000:01
  Attempting uninstall: openai
    Found existing installation: openai 2.7.1
    Uninstalling openai-2.7.1:
      Successfully uninstalled openai-2.7.1


In [ ]:
import os

client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [92]:
import pandas as pd
import json
from tqdm import tqdm

In [94]:
# ===== Hàm đánh giá summarization với GPT-4o-mini =====
def evaluate_summarization_gpt4o_vi(document, reference, prediction):
    system_prompt = """
Bạn là chuyên gia đánh giá tóm tắt văn bản.
Đánh giá 4 tiêu chí (1–10):
1️⃣ Fluency: Lưu loát & mạch lạc.
2️⃣ Relevance: Liên quan & Hữu ích.
3️⃣ Faithfulness: Độ trung thực với nội dung gốc.
4️⃣ Conciseness: Ngắn gọn, súc tích.

Phải phản hồi **JSON hợp lệ**:
{
  "fluency": <điểm>,
  "relevance": <điểm>,
  "faithfulness": <điểm>,
  "conciseness": <điểm>,
  "average": <điểm>
}
"""
    evaluation_prompt = f"""
Văn bản gốc:
{document}

Tóm tắt chuẩn (reference):
{reference}

Tóm tắt mô hình (prediction):
{prediction}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": evaluation_prompt}
            ],
            response_format={"type": "json_object"}
        )
        raw_text = response.choices[0].message.content.strip()
        result = json.loads(raw_text)
    except Exception as e:
        print("⚠️ Lỗi API hoặc lỗi JSON:", e)
        result = {k: None for k in ["fluency","relevance","faithfulness","conciseness","average"]}
    return result

In [95]:
# ===== Đánh giá tập 50 tóm tắt =====
scores = {"fluency": [], "relevance": [], "faithfulness": [], "conciseness": [], "average": []}

for doc, ref, pred in tqdm(zip(test_dataset["document"][:50],
                               test_dataset["summary"][:50],
                               pred_summaries), total=50):
    res = evaluate_summarization_gpt4o_vi(doc, ref, pred)
    for key in scores:
        scores[key].append(res[key])



100%|██████████| 50/50 [01:13<00:00,  1.47s/it]


In [96]:
# ===== Chuyển sang DataFrame =====
df_scores = pd.DataFrame(scores)
# ===== Tính trung bình toàn bộ =====
avg_scores = df_scores.mean()
print("\nAverage | Fluency: {:.2f} | Relevance: {:.2f} | Faithfulness: {:.2f} | Conciseness: {:.2f} | Avg: {:.2f}"
      .format(avg_scores['fluency'], avg_scores['relevance'], avg_scores['faithfulness'],
              avg_scores['conciseness'], avg_scores['average']))

Sample  1 | Fluency: 6.00 | Relevance: 7.00 | Faithfulness: 5.00 | Conciseness: 4.00 | Avg: 5.50
Sample  2 | Fluency: 6.00 | Relevance: 7.00 | Faithfulness: 5.00 | Conciseness: 4.00 | Avg: 5.50
Sample  3 | Fluency: 7.00 | Relevance: 9.00 | Faithfulness: 8.00 | Conciseness: 6.00 | Avg: 7.50
Sample  4 | Fluency: 6.00 | Relevance: 7.00 | Faithfulness: 5.00 | Conciseness: 6.00 | Avg: 6.00
Sample  5 | Fluency: 5.00 | Relevance: 6.00 | Faithfulness: 4.00 | Conciseness: 3.00 | Avg: 4.50
Sample  6 | Fluency: 8.00 | Relevance: 9.00 | Faithfulness: 7.00 | Conciseness: 6.00 | Avg: 7.50
Sample  7 | Fluency: 7.00 | Relevance: 6.00 | Faithfulness: 5.00 | Conciseness: 6.00 | Avg: 6.00
Sample  8 | Fluency: 7.00 | Relevance: 9.00 | Faithfulness: 8.00 | Conciseness: 6.00 | Avg: 7.50
Sample  9 | Fluency: 8.00 | Relevance: 9.00 | Faithfulness: 7.00 | Conciseness: 6.00 | Avg: 7.50
Sample 10 | Fluency: 8.00 | Relevance: 9.00 | Faithfulness: 7.00 | Conciseness: 6.00 | Avg: 7.50
Sample 11 | Fluency: 8.00 | Re

In [24]:

def generate_synthetic_summary(title, document):
    prompt = f"""
Bạn là mô hình tóm tắt tiếng Việt.
Hãy viết bản tóm tắt ngắn, rõ ràng, không bỏ ý.

Tiêu đề: {title}

Văn bản:
{document}

Yêu cầu: Tóm tắt 3–5 câu, không thêm thông tin mới.
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Bạn là mô hình tóm tắt tiếng Việt."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


In [25]:
import pandas as pd

synthetic_title = []
synthetic_document = []
synthetic_summary = []

# Lấy 100 mẫu đầu từ train
samples = dataset["train"].select(range(500))  # chọn 100 mẫu

for sample in samples:
    title = sample["title"]
    document = sample["document"]
    
    # Gọi hàm tạo summary
    new_summary = generate_synthetic_summary(title, document)

    synthetic_title.append(title)
    synthetic_document.append(document)
    synthetic_summary.append(new_summary)

# Tạo DataFrame
synthetic_df = pd.DataFrame({
    "title": synthetic_title,
    "document": synthetic_document,
    "summary": synthetic_summary
})

print(synthetic_df.head())


                                               title  \
0   3 câu chuyện truyền cảm hứng chiến thắng ung thư   
1        TP.HCM siết nhà cao tầng các quận trung tâm   
2  Sửng sốt với vẻ đẹp của ngôi nhà mê cung kết h...   
3  3 tiêu chí vàng để làm đẹp bằng phương pháp că...   
4  Tốt nghiệp thạc sĩ, chàng trai Quảng Ngãi quỳ ...   

                                            document  \
0  Cơn bạo bệnh bất ngờ ập đếnCứ nhắc tới hai chữ...   
1  Trong báo cáo gửi UBND TPHCM về đề án “Xây dựn...   
2  Dự án cải tạo ngôi nhà kiêm thư viện, phòng kh...   
3  Cùng lưu ý 3 tiêu chí vàng để căng chỉ tạo col...   
4  Khoảnh khắc Hoàng Anh quỳ lạy công ơn của mẹ g...   

                                             summary  
0  Bài viết kể về hành trình chiến thắng ung thư ...  
1  Sở Xây dựng TP.HCM đã báo cáo UBND về việc hạn...  
2  Ngôi nhà Guha ở Indonesia là một dự án cải tạo...  
3  Để thực hiện phương pháp căng chỉ tạo collagen...  
4  Nguyễn Hoàng Anh, chàng trai từ Quảng Ngãi, đã..

In [26]:
from datasets import Dataset

synthetic_dataset = Dataset.from_pandas(synthetic_df)

print("Synthetic samples:", len(synthetic_dataset))


Synthetic samples: 500


In [27]:
tokenized_synthetic = synthetic_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [28]:
from datasets import concatenate_datasets

aug_train_dataset = concatenate_datasets([
    tokenized_dataset["train"],
    tokenized_synthetic
])

print("Train gốc:", len(tokenized_dataset["train"]))
print("Synthetic:", len(tokenized_synthetic))
print("Tổng train augmented:", len(aug_train_dataset))


Train gốc: 2388
Synthetic: 500
Tổng train augmented: 2888


In [29]:
training_args_2 = Seq2SeqTrainingArguments(
    output_dir="./output_augmented",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,

    eval_strategy="steps",
    eval_steps=100,                  
    logging_strategy="steps",
    logging_steps=50,               

    save_strategy="no",              
    predict_with_generate=True,    
    report_to="none",                
    fp16=True
)



In [30]:

trainer_2 = Seq2SeqTrainer(
    model=model,
    args=training_args_2,
    train_dataset=aug_train_dataset,
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_48/561420147.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_2 = Seq2SeqTrainer(


In [31]:
trainer_2.train()




/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.628400,1.678730
200,1.508400,1.678709
300,1.430600,1.669907
400,1.343300,1.669558
500,1.358200,1.670501


TrainOutput(global_step=543, training_loss=1.4552802407280516, metrics={'train_runtime': 2153.6437, 'train_samples_per_second': 4.023, 'train_steps_per_second': 0.252, 'total_flos': 1.02973827796992e+16, 'train_loss': 1.4552802407280516, 'epoch': 3.0})

In [32]:
trainer_2.save_model("./model_summarization_augmented")
tokenizer.save_pretrained("./model_summarization_augmented")

('./model_summarization_augmented/tokenizer_config.json',
 './model_summarization_augmented/special_tokens_map.json',
 './model_summarization_augmented/spiece.model',
 './model_summarization_augmented/added_tokens.json',
 './model_summarization_augmented/tokenizer.json')

In [34]:
# ===== Load model 1 =====
model1 = AutoModelForSeq2SeqLM.from_pretrained("./output_loss_only")
tokenizer1 = AutoTokenizer.from_pretrained("./output_loss_only")

model1.eval()

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [35]:
# ===== Load model 2 =====
model2 = AutoModelForSeq2SeqLM.from_pretrained("./model_summarization_augmented")
tokenizer2 = AutoTokenizer.from_pretrained("./model_summarization_augmented")

model2.eval()

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [39]:
# ===== Hàm summarize wrapper =====
def summarize_model1(text, max_input_length=1024, max_output_length=200, min_output_length=80):
    inputs = tokenizer1(text, return_tensors="pt", truncation=True, max_length=max_input_length)
    output_ids = model1.generate(
        **inputs,
        max_length=max_output_length,
        min_length=min_output_length,
        num_beams=5,
        no_repeat_ngram_size=3,
        length_penalty=1.0,
        early_stopping=True
    )
    return tokenizer1.decode(output_ids[0], skip_special_tokens=True)

def summarize_model2(text, max_input_length=1024, max_output_length=200, min_output_length=80):
    inputs = tokenizer2(text, return_tensors="pt", truncation=True, max_length=max_input_length)
    output_ids = model2.generate(
        **inputs,
        max_length=max_output_length,
        min_length=min_output_length,
        num_beams=5,
        no_repeat_ngram_size=3,
        length_penalty=1.0,
        early_stopping=True
    )
    return tokenizer2.decode(output_ids[0], skip_special_tokens=True)

In [40]:
# ===== Test inference =====
text = """
Chấm miếng ăn cắn dở vào bát nước chung không chỉ mất lịch sự, mà còn có thể lây truyền 10.000 vi khuẩn cho người khác, theo nghiên cứu của Đại học Clemson (Mỹ).
Trong các bữa ăn, hành động chấm hai lần (double-dipping, nhúng miếng thức ăn đã cắn dở vào bát chấm chung) luôn là chủ đề gây tranh cãi. Nhiều người coi đây là thói quen mất vệ sinh, số khác lại cho rằng sự lo ngại này là thái quá.
Nghiên cứu năm 2008 trên tạp chí An toàn thực phẩm (Journal of Food Safety) đã chứng minh hành vi này rất nguy hại. Nhóm nghiên cứu do tiến sĩ Paul Dawson, Đại học Clemson chủ trì đã dùng bánh quy chấm vào các loại sốt khác nhau, bao gồm nước vô trùng, sốt salsa, sốt phô mai và siro chocolate.
Kết quả cho thấy vi khuẩn lây lan ở mức độ đáng báo động. Trung bình, chỉ 6 lần nhúng (miếng bánh đã cắn dở) đã chuyển khoảng 10.000 vi khuẩn từ miệng người ăn vào nước chấm.

"""

In [42]:
print("Model1 summary:", summarize_model1(text))


Model1 summary: Theo nghiên cứu của Đại học Clemson (Mỹ), chấm miếng ăn đã cắn dở vào bát nước chung không chỉ mất lịch sự, mà còn có thể lây truyền 10.000 vi khuẩn từ miệng người ăn vào nước chấm, theo nghiên cứu năm 2008 trên tạp chí An toàn thực phẩm (Food Safety), gây nguy hiểm cho người khác.


In [43]:
print("Model2 summary:", summarize_model2(text))

Model2 summary: Hành động chấm miếng ăn đã cắn dở vào bát nước chung không chỉ mất lịch sự, mà còn có thể lây truyền 10.000 vi khuẩn cho người khác, theo nghiên cứu của Đại học Clemson (Mỹ). Hành động này có thể gây nguy hiểm cho người ăn, theo một nghiên cứu năm 2008 trên tạp chí An toàn thực phẩm (Journal of Food Safety).


In [45]:
import pandas as pd

# Giả sử test_dataset là Dataset của HF, đã có 'document' và 'summary'
test_dataset = dataset['test']  # thêm dòng này trước khi dùng

num_samples = 50
docs = test_dataset["document"][:num_samples]
refs = test_dataset["summary"][:num_samples]




In [46]:
# ===== Dự đoán với Model 1 =====
pred_summaries_model1 = []
for doc in docs:
    summary = summarize_model1(doc, max_input_length=1024, max_output_length=256, min_output_length=120)
    pred_summaries_model1.append(summary)

In [47]:
# ===== Dự đoán với Model 2 =====
pred_summaries_model2 = []
for doc in docs:
    summary = summarize_model2(doc, max_input_length=1024, max_output_length=256, min_output_length=120)
    pred_summaries_model2.append(summary)


In [48]:
# ===== Tạo DataFrame chuẩn bị đánh giá =====
df_model1 = pd.DataFrame({
    "document": docs,
    "reference_summary": refs,
    "pred_summary_model1": pred_summaries_model1
})

In [49]:

df_model2 = pd.DataFrame({
    "document": docs,
    "reference_summary": refs,
    "pred_summary_model2": pred_summaries_model2
})

In [50]:


# In thử 3 dòng đầu
print(df_model1.head(3))
print(df_model2.head(3))

                                            document  \
0  Các VĐV tranh tài ở 48 bộ huy chương, gồm 26 b...   
1  Bệnh viện K cho biết, ung thư thận ở giai đoạn...   
2  Lương cao kèm điều kiện "đánh đố"Người lao độn...   

                                   reference_summary  \
0  Hơn 350 vận động viên (VĐV) tham dự giải vô đị...   
1  Tiểu ra máu có thể cảnh báo nhiều bệnh lý, tro...   
2  Các doanh nghiệp ở TPHCM có nhu cầu tuyển gần ...   

                                 pred_summary_model1  
0  Trong khuôn khổ giải vô địch Vovinam châu Á 20...  
1  Theo các chuyên gia, ung thư thận ở giai đoạn ...  
2  Theo thống kê trực tuyến của Trung tâm Dịch vụ...  
                                            document  \
0  Các VĐV tranh tài ở 48 bộ huy chương, gồm 26 b...   
1  Bệnh viện K cho biết, ung thư thận ở giai đoạn...   
2  Lương cao kèm điều kiện "đánh đố"Người lao độn...   

                                   reference_summary  \
0  Hơn 350 vận động viên (VĐV) tham dự giải vô đị

In [54]:
import evaluate

bertscore = evaluate.load("bertscore")

In [58]:
rouge = evaluate.load("rouge")

In [61]:
# ===== Số lượng sample test =====
num_samples = 50
refs = test_dataset["summary"][:num_samples]

# ===== ROUGE =====
rouge_model1 = rouge.compute(predictions=pred_summaries_model1, references=refs)
rouge_model2 = rouge.compute(predictions=pred_summaries_model2, references=refs)

# Trung bình ROUGE F1
avg_rouge_model1 = {k: v for k, v in rouge_model1.items()}
avg_rouge_model2 = {k: v for k, v in rouge_model2.items()}

# ===== BERTScore =====
bertscore_model1 = bertscore.compute(predictions=pred_summaries_model1, references=refs, lang="vi")
bertscore_model2 = bertscore.compute(predictions=pred_summaries_model2, references=refs, lang="vi")

# Trung bình BERTScore: chuyển tất cả về float
avg_bertscore_model1 = {k: float(sum(v))/len(v) for k, v in bertscore_model1.items() if k in ["precision","recall","f1"]}
avg_bertscore_model2 = {k: float(sum(v))/len(v) for k, v in bertscore_model2.items() if k in ["precision","recall","f1"]}


In [62]:
# ===== Tạo bảng so sánh =====
df_compare = pd.DataFrame({
    "Metric": ["ROUGE-1", "ROUGE-2", "ROUGE-L", "BERTScore Precision", "BERTScore Recall", "BERTScore F1"],
    "Model 1": [
        avg_rouge_model1.get("rouge1", 0),
        avg_rouge_model1.get("rouge2", 0),
        avg_rouge_model1.get("rougeL", 0),
        avg_bertscore_model1["precision"],
        avg_bertscore_model1["recall"],
        avg_bertscore_model1["f1"]
    ],
    "Model 2": [
        avg_rouge_model2.get("rouge1", 0),
        avg_rouge_model2.get("rouge2", 0),
        avg_rouge_model2.get("rougeL", 0),
        avg_bertscore_model2["precision"],
        avg_bertscore_model2["recall"],
        avg_bertscore_model2["f1"]
    ]
})

print(df_compare)

                Metric   Model 1   Model 2
0              ROUGE-1  0.441434  0.429823
1              ROUGE-2  0.213093  0.202547
2              ROUGE-L  0.277753  0.273233
3  BERTScore Precision  0.685912  0.682779
4     BERTScore Recall  0.749460  0.754380
5         BERTScore F1  0.716103  0.716505


In [65]:
import json
import pandas as pd
from tqdm import tqdm

In [66]:
def evaluate_summarization_gpt4o_vi(document, reference, prediction):
    system_prompt = """
Bạn là chuyên gia đánh giá tóm tắt văn bản.
Đánh giá 4 tiêu chí (1–10):
1️⃣ Fluency: Lưu loát & mạch lạc.
2️⃣ Relevance: Liên quan & hữu ích.
3️⃣ Faithfulness: Độ trung thực với nội dung gốc.
4️⃣ Conciseness: Ngắn gọn, súc tích.

Phải phản hồi **JSON hợp lệ**:
{
  "fluency": <điểm>,
  "relevance": <điểm>,
  "faithfulness": <điểm>,
  "conciseness": <điểm>,
  "average": <điểm>
}
"""
    evaluation_prompt = f"""
Văn bản gốc:
{document}

Tóm tắt chuẩn (reference):
{reference}

Tóm tắt mô hình (prediction):
{prediction}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": evaluation_prompt}
            ],
            response_format={"type": "json_object"}
        )
        # Lấy JSON từ GPT
        raw_text = response.choices[0].message.content.strip()
        result = json.loads(raw_text)
        # Ép kiểu float
        for k in result:
            result[k] = float(result[k])
    except Exception as e:
        print("⚠️ Lỗi API hoặc lỗi JSON:", e)
        result = {k: None for k in ["fluency","relevance","faithfulness","conciseness","average"]}
    return result

In [67]:
num_samples = 50
docs = test_dataset["document"][:num_samples]
refs = test_dataset["summary"][:num_samples]

In [68]:
evals_model1 = []
evals_model2 = []

print("🔹 Evaluating Model 1")
for i in tqdm(range(num_samples)):
    eval_result = evaluate_summarization_gpt4o_vi(
        document=docs[i],
        reference=refs[i],
        prediction=pred_summaries_model1[i]
    )
    evals_model1.append(eval_result)

🔹 Evaluating Model 1


100%|██████████| 50/50 [01:11<00:00,  1.43s/it]


In [69]:
for i in tqdm(range(num_samples), desc="Evaluating Model 2"):
    eval_result = evaluate_summarization_gpt4o_vi(
        document=docs[i],
        reference=refs[i],
        prediction=pred_summaries_model2[i]
    )
    evals_model2.append(eval_result)

Evaluating Model 2: 100%|██████████| 50/50 [01:06<00:00,  1.34s/it]


In [75]:
# Chuyển sang DataFrame
df_gpt_eval = pd.DataFrame({
    "document": docs,
    "reference_summary": refs,
    "pred_model1": pred_summaries_model1,
    "pred_model2": pred_summaries_model2,
    "eval_model1": evals_model1,
    "eval_model2": evals_model2
})

In [76]:
# Tách riêng các cột điểm cho dễ tính trung bình
for criterion in ["fluency","relevance","faithfulness","conciseness","average"]:
    df_gpt_eval[f"{criterion}_model1"] = [e[criterion] for e in evals_model1]
    df_gpt_eval[f"{criterion}_model2"] = [e[criterion] for e in evals_model2]


In [77]:
# Tính điểm trung bình tổng thể cho 2 model
avg_scores_model1 = df_gpt_eval[[f"{c}_model1" for c in ["fluency","relevance","faithfulness","conciseness","average"]]].mean()
avg_scores_model2 = df_gpt_eval[[f"{c}_model2" for c in ["fluency","relevance","faithfulness","conciseness","average"]]].mean()

In [78]:
df_avg_gpt = pd.DataFrame({
    "Metric": ["Fluency","Relevance","Faithfulness","Conciseness","Average"],
    "Model 1": avg_scores_model1.values,
    "Model 2": avg_scores_model2.values
})

print(df_avg_gpt)

         Metric  Model 1  Model 2
0       Fluency     6.44     7.42
1     Relevance     7.18     8.12
2  Faithfulness     5.66     6.84
3   Conciseness     4.92     6.10
4       Average     6.05     7.12


In [80]:
# ✅ 1. Đăng nhập Hugging Face
from huggingface_hub import login
login(new_session=False)  #

In [ ]:
trainer.push_to_hub("letri/vit5_text summarization")
trainer.push_to_hub("letri/vit5_text summarization_Synthetic Data")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            